<a href="https://www.kaggle.com/code/trakiet/duachuot?scriptVersionId=270824718" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/user-course-v4/user_course_v4.csv
/kaggle/input/df-user-video-final/part-00000-8eb8369b-ec66-434a-9e3c-608414a06b1f-c000.csv
/kaggle/input/mooccubex-dataset/course-teacher.txt
/kaggle/input/mooccubex-dataset/course.json
/kaggle/input/mooccubex-dataset/paper.json
/kaggle/input/mooccubex-dataset/reply.json
/kaggle/input/mooccubex-dataset/concept-comment.txt
/kaggle/input/mooccubex-dataset/user-video.json
/kaggle/input/mooccubex-dataset/course-comment.txt
/kaggle/input/mooccubex-dataset/concept-problem.txt
/kaggle/input/mooccubex-dataset/user-reply.txt
/kaggle/input/mooccubex-dataset/school.json
/kaggle/input/mooccubex-dataset/translated_reply.json
/kaggle/input/mooccubex-dataset/video.json
/kaggle/input/mooccubex-dataset/concept-video.txt
/kaggle/input/mooccubex-dataset/other.json
/kaggle/input/mooccubex-dataset/course-school.txt
/kaggle/input/mooccubex-dataset/problem.json
/kaggle/input/mooccubex-dataset/user-comment.txt
/kaggle/input/mooccubex-dataset/user.json
/kaggle/in

In [2]:
BASE_DIR = '/kaggle/input/mooccubex-dataset/'
df_user_course_path = '/kaggle/input/user-course-v4/user_course_v4.csv'
df_user_video_final_path = '/kaggle/input/df-user-video-final/part-00000-8eb8369b-ec66-434a-9e3c-608414a06b1f-c000.csv'

In [3]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt
from os import truncate
from pyspark.sql.functions import explode, col, sum, from_unixtime, hour, dayofweek, count, round, when, isnull, split, avg, regexp_replace
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
  .appName("reply_user_video_analysis") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/25 17:24:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!ls {BASE_DIR}

comment-reply.txt    course-school.txt	      translated_reply.json
concept-comment.txt  course-teacher.txt       user-comment.txt
concept.json	     exercise-problem.txt     user.json
concept-other.txt    other.json		      user-problem.json
concept-paper.txt    paper.json		      user-reply.txt
concept-problem.txt  problem.json	      user-video.json
concept-reply.json   reply.json		      user-xiaomu.json
concept-video.txt    reply-reply.txt	      video_id-ccid.txt
course-comment.txt   school.json	      video.json
course-field.json    teacher.json
course.json	     translated_comment.json


In [5]:
df_user_video=spark.read.json(BASE_DIR + "/user-video.json")
# df_user_video.show()

In [47]:
df_user_video.printSchema()

root
 |-- seq: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- segment: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end_point: double (nullable = true)
 |    |    |    |    |-- local_start_time: long (nullable = true)
 |    |    |    |    |-- speed: double (nullable = true)
 |    |    |    |    |-- start_point: double (nullable = true)
 |    |    |-- video_id: string (nullable = true)
 |-- user_id: string (nullable = true)



In [7]:
from pyspark.sql.functions import col, explode, from_json, expr
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

# Giải nén `seq` để mỗi dòng có một `video_id`
df_flatten = df_user_video.withColumn("seq", explode(col("seq")))

# Giải nén tiếp `segment` để mỗi dòng là một đoạn xem video
df_flatten = df_flatten.withColumn("segment", explode(col("seq.segment")))

# Chọn các cột cần thiết
df_flatten = df_flatten.select(
    col("user_id"),
    col("seq.video_id").alias("video_id"),
    col("segment.start_point"),
    col("segment.end_point"),
    col("segment.speed"),
    col("segment.local_start_time")
)
# df_flatten.show(truncate=True)

In [8]:
from pyspark.sql.functions import from_unixtime

df_flatten = df_flatten.withColumn("local_start_time", from_unixtime("local_start_time", "yyyy-MM-dd HH:mm:ss"))
df_flatten.show(truncate=False)

+-------+---------+-----------+---------+-----+-------------------+
|user_id|video_id |start_point|end_point|speed|local_start_time   |
+-------+---------+-----------+---------+-----+-------------------+
|U_112  |V_1395633|130.0      |190.0    |1.0  |2020-05-02 14:52:24|
|U_112  |V_1395633|220.0      |250.0    |1.0  |2020-05-02 14:53:54|
|U_112  |V_1395633|478.8      |508.8    |1.0  |2020-05-02 16:35:59|
|U_112  |V_1395633|655.1      |692.55   |1.25 |2020-05-02 16:38:34|
|U_112  |V_1395635|135.0      |170.0    |1.0  |2020-05-02 16:47:25|
|U_112  |V_1395636|128.3      |180.8    |1.5  |2020-05-02 16:51:18|
|U_112  |V_1395639|100.0      |106.25   |1.25 |2020-05-02 17:03:00|
|U_112  |V_1395639|180.0      |186.25   |1.25 |2020-05-02 17:04:05|
|U_112  |V_6210799|9.0        |22.25    |1.25 |2020-09-25 05:44:16|
|U_112  |V_6210799|28.0       |271.5    |1.25 |2020-09-25 05:44:31|
|U_112  |V_6210799|279.0      |331.5    |1.5  |2020-09-25 05:47:21|
|U_112  |V_6210799|346.0      |356.0    |1.5  |2

In [9]:
df_flatten = df_flatten.filter(col("start_point") <= col("end_point"))
df_flatten.show()

+-------+---------+-----------+---------+-----+-------------------+
|user_id| video_id|start_point|end_point|speed|   local_start_time|
+-------+---------+-----------+---------+-----+-------------------+
|  U_112|V_1395633|      130.0|    190.0|  1.0|2020-05-02 14:52:24|
|  U_112|V_1395633|      220.0|    250.0|  1.0|2020-05-02 14:53:54|
|  U_112|V_1395633|      478.8|    508.8|  1.0|2020-05-02 16:35:59|
|  U_112|V_1395633|      655.1|   692.55| 1.25|2020-05-02 16:38:34|
|  U_112|V_1395635|      135.0|    170.0|  1.0|2020-05-02 16:47:25|
|  U_112|V_1395636|      128.3|    180.8|  1.5|2020-05-02 16:51:18|
|  U_112|V_1395639|      100.0|   106.25| 1.25|2020-05-02 17:03:00|
|  U_112|V_1395639|      180.0|   186.25| 1.25|2020-05-02 17:04:05|
|  U_112|V_6210799|        9.0|    22.25| 1.25|2020-09-25 05:44:16|
|  U_112|V_6210799|       28.0|    271.5| 1.25|2020-09-25 05:44:31|
|  U_112|V_6210799|      279.0|    331.5|  1.5|2020-09-25 05:47:21|
|  U_112|V_6210799|      346.0|    356.0|  1.5|2

In [48]:
df_flatten.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- video_id: string (nullable = true)
 |-- start_point: double (nullable = true)
 |-- end_point: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- local_start_time: string (nullable = true)



In [10]:
df_course = spark.read.json(BASE_DIR + '/course.json')
df_video_course_map = (
    df_course
    .withColumn(
        "video_resource",
        F.filter(F.col("resource"), lambda x: x["resource_id"].startswith("V_"))
    )
    .withColumn("video_struct", F.explode_outer(F.col("video_resource")))
    .select(
        F.col("id").alias("course_id"),
        F.col("video_struct.resource_id").alias("video_id")
    )
)

df_video_course_map.show()

+---------+--------+
|course_id|video_id|
+---------+--------+
| C_584313|   V_849|
| C_584313|   V_850|
| C_584313|   V_851|
| C_584313|   V_857|
| C_584313|   V_859|
| C_584313|   V_861|
| C_584313|   V_862|
| C_584313|   V_867|
| C_584313|   V_868|
| C_584313|   V_869|
| C_584313|   V_870|
| C_584313|   V_871|
| C_584313|   V_877|
| C_584313|   V_878|
| C_584313|   V_879|
| C_584313|   V_880|
| C_584313|   V_881|
| C_584313|   V_883|
| C_584313|   V_884|
| C_584313|   V_885|
+---------+--------+
only showing top 20 rows



In [11]:
# df_video_course_map.coalesce(1).write.mode("overwrite").option("header", "true").csv("/kaggle/working/df_video_course_map/")

In [12]:
# import shutil

# input_dir = "/kaggle/working/df_video_course_map"
# output_zip = "/kaggle/working/df_video_course_map"

# shutil.make_archive(output_zip, 'zip', input_dir)
# print("✅ Đã nén file thành df_video_course_map.zip")


✅ Đã nén file thành df_video_course_map.zip


In [13]:
video_course_count = df_video_course_map.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_video_course_map.columns
])
video_course_count.show(truncate=False)

+---------+--------+
|course_id|video_id|
+---------+--------+
|0        |11      |
+---------+--------+



In [54]:
df_video = spark.read.json(BASE_DIR + "/video.json")
from pyspark.sql.functions import array_max
df_video = (
    df_video
    .select(
        "ccid", 
        array_max("end").alias("video_duration")
    )
)
df_video.show(truncate=False)

+--------------------------------+--------------+
|ccid                            |video_duration|
+--------------------------------+--------------+
|0001603F826A3DDA9C33DC5901307461|509.095       |
|0003DB14A14A53279C33DC5901307461|367.336       |
|0004A5C6F07E369D9C33DC5901307461|492.315       |
|00059EBD1371A6959C33DC5901307461|630.757       |
|0005D1DC01B4EF8F9C33DC5901307461|408.75        |
|0007919ED7652A779C33DC5901307461|506.57        |
|0007C4A5229EE3CF9C33DC5901307461|1190.78       |
|0008C8CC056F4EAC9C33DC5901307461|130.309       |
|000948D39EA6B0A69C33DC5901307461|421.32        |
|0009510AE8068CBA9C33DC5901307461|622.68        |
|000A1158B61C13B79C33DC5901307461|1281.36       |
|000A40E1D226AEE09C33DC5901307461|255.465       |
|000A5EE0C695E8EF9C33DC5901307461|553.8         |
|000BD50F6730291F9C33DC5901307461|1328.56       |
|000E08952A010F3D9C33DC5901307461|590.0         |
|000F83F0945DC9D79C33DC5901307461|659.68        |
|0012B03A05345F229C33DC5901307461|17.18         |


In [52]:
df_video_id_ccid = spark.read.text(BASE_DIR + '/video_id-ccid.txt')
# Parse mapping file
df_video_id_ccid_split = df_video_id_ccid.select(
    split(col("value"), "\t")[0].alias("video_id"),
    split(col("value"), "\t")[1].alias("ccid")
)
df_video_id_ccid_split.show(truncate=False)

+---------+--------------------------------+
|video_id |ccid                            |
+---------+--------------------------------+
|V_234845 |0000363DB5B6E0869C33DC5901307461|
|V_234876 |0000363DB5B6E0869C33DC5901307461|
|V_234907 |0000363DB5B6E0869C33DC5901307461|
|V_293392 |0000363DB5B6E0869C33DC5901307461|
|V_293445 |0000363DB5B6E0869C33DC5901307461|
|V_293494 |0000363DB5B6E0869C33DC5901307461|
|V_627142 |0000363DB5B6E0869C33DC5901307461|
|V_944346 |0000363DB5B6E0869C33DC5901307461|
|V_944377 |0000363DB5B6E0869C33DC5901307461|
|V_944408 |0000363DB5B6E0869C33DC5901307461|
|V_1386166|0000363DB5B6E0869C33DC5901307461|
|V_1479635|0000363DB5B6E0869C33DC5901307461|
|V_1494506|0000363DB5B6E0869C33DC5901307461|
|V_1494536|0000363DB5B6E0869C33DC5901307461|
|V_1494566|0000363DB5B6E0869C33DC5901307461|
|V_1494614|0000363DB5B6E0869C33DC5901307461|
|V_1494625|0000363DB5B6E0869C33DC5901307461|
|V_1494656|0000363DB5B6E0869C33DC5901307461|
|V_1570352|0000363DB5B6E0869C33DC5901307461|
|V_2070578

In [ ]:
video_info = df_video_id_ccid_split.join(df_video, on=['ccid'], how='left')
video_info.show(truncate=False)

In [44]:
from pyspark.sql import functions as F
video_info.select(F.countDistinct("video_id").alias("num_distinct_video_id")).show()

+---------------------+
|num_distinct_video_id|
+---------------------+
|              2798892|
+---------------------+



In [46]:
video_info_count = video_info.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in video_info.columns
])
video_info_count.show(truncate=False)

+----+--------+--------------+
|ccid|video_id|video_duration|
+----+--------+--------------+
|0   |0       |1482802       |
+----+--------+--------------+



In [45]:
df_course_video_info = df_video_course_map.join(video_info, on='video_id', how = 'left')
df_course_video_info.show()

+---------+---------+--------------------+--------------+
| video_id|course_id|                ccid|video_duration|
+---------+---------+--------------------+--------------+
|V_1224994| C_932180|6F1E041C820CB2F59...|       559.091|
|V_1224998| C_932180|5B39CFD3815B4D2B9...|       511.922|
|V_1225002| C_932180|3213B53265D17AB69...|       767.554|
|V_2961778| C_676975|429EB7C97CA852AF9...|          NULL|
|V_2962669| C_676975|00B886C53A82289F9...|          NULL|
| V_471505| C_696863|76168D418AC9437F9...|          NULL|
| V_471511| C_696863|DE965C5A77AFC35D9...|          NULL|
| V_471513| C_696863|DD9A75726B972E629...|          NULL|
| V_471524| C_696863|37D7ACF856036DF39...|          NULL|
| V_471536| C_696863|1EBB825CBB1FC9359...|          NULL|
| V_471538| C_696863|1F6E41380D563BB69...|          NULL|
| V_471539| C_696863|5665F11FD1082FDB9...|          NULL|
| V_471555| C_696863|2A2893358F5E832C9...|          NULL|
| V_471561| C_696863|BF0EA319F07D4A269...|          NULL|
|  V_74847| C_

In [42]:
from pyspark.sql import functions as F
df_course_video_info.select(F.countDistinct("video_id").alias("num_distinct_video_id")).show()

+---------------------+
|num_distinct_video_id|
+---------------------+
|               203718|
+---------------------+



In [20]:
df_course_video_info.count()

203729

In [21]:
df_course_video_info_count = df_course_video_info.count()

In [22]:
from pyspark.sql.functions import col, count, when, isnull
# 1. Lấy tổng số dòng
total_row_count = df_course_video_info.count()

# 2. Tạo danh sách các biểu thức (expressions) để tính % null cho mỗi cột
null_percentage_exprs = [
    ((count(when(isnull(c), c)) / total_row_count) * 100).alias(f"perc_null_{c}")
    for c in df_course_video_info.columns
]

# 3. Select và show kết quả
# Kết quả sẽ là một hàng duy nhất hiển thị % null cho tất cả các cột
df_course_video_info.select(null_percentage_exprs).show()

+--------------------+-------------------+-----------------+------------------------+
|  perc_null_video_id|perc_null_course_id|   perc_null_ccid|perc_null_video_duration|
+--------------------+-------------------+-----------------+------------------------+
|0.005399329501445548|                0.0|57.89161091449916|       57.89161091449916|
+--------------------+-------------------+-----------------+------------------------+



In [23]:
df_user_course = spark.read.csv(df_user_course_path, header=True, inferSchema=True)
df_user_course.show()

+---------+---------+-----------+----------+----------+-------------+-------------------+----------+
|  user_id|course_id|certificate|start_date|  end_date|duration_days|        enroll_time|remain_day|
+---------+---------+-----------+----------+----------+-------------+-------------------+----------+
|  U_10000|C_2033958|        1.0|2020-09-04|2020-12-31|        118.0|2020-10-27 09:07:30|      65.0|
|U_1000129|C_1925903|        1.0|2020-08-12|2020-12-31|        141.0|2020-09-06 14:37:17|     116.0|
|U_1000129|C_1992970|        1.0|2020-08-27|2020-12-31|        126.0|2020-09-06 14:45:37|     116.0|
|U_1000129| C_680884|        1.0|2020-09-01|2020-12-31|        121.0|2020-10-16 10:41:09|      76.0|
|U_1000342| C_697791|        0.0|2020-01-20|2020-07-31|        193.0|2020-02-28 10:27:22|     154.0|
|U_1000663|C_2328510|        0.0|2020-11-18|2021-02-28|        102.0|2020-11-27 22:41:36|      93.0|
| U_100071| C_696601|        1.0|2019-12-25|2020-06-30|        188.0|2020-04-15 08:51:39|  

In [53]:
df_user_course.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- course_id: string (nullable = true)
 |-- certificate: double (nullable = true)
 |-- start_date: date (nullable = true)
 |-- end_date: date (nullable = true)
 |-- duration_days: double (nullable = true)
 |-- enroll_time: timestamp (nullable = true)
 |-- remain_day: double (nullable = true)



In [24]:
uc_count = df_user_course.select([
    (F.count(F.when(F.col(c).isNull(), 1))).alias(c) for c in df_user_course.columns
])
uc_count.show(truncate=False)

+-------+---------+-----------+----------+--------+-------------+-----------+----------+
|user_id|course_id|certificate|start_date|end_date|duration_days|enroll_time|remain_day|
+-------+---------+-----------+----------+--------+-------------+-----------+----------+
|0      |0        |4939141    |4939141   |4939141 |4939141      |0          |4939141   |
+-------+---------+-----------+----------+--------+-------------+-----------+----------+



In [25]:
# 1. Lấy tổng số dòng
total_row_count = df_user_course.count()

# 2. Tạo danh sách các biểu thức (expressions) để tính % null cho mỗi cột
null_percentage_exprs = [
    ((count(when(isnull(c), c)) / total_row_count) * 100).alias(f"perc_null_{c}")
    for c in df_user_course.columns
]

# 3. Select và show kết quả
# Kết quả sẽ là một hàng duy nhất hiển thị % null cho tất cả các cột
df_user_course.select(null_percentage_exprs).show()

+-----------------+-------------------+---------------------+--------------------+------------------+-----------------------+---------------------+--------------------+
|perc_null_user_id|perc_null_course_id|perc_null_certificate|perc_null_start_date|perc_null_end_date|perc_null_duration_days|perc_null_enroll_time|perc_null_remain_day|
+-----------------+-------------------+---------------------+--------------------+------------------+-----------------------+---------------------+--------------------+
|              0.0|                0.0|    70.47542227327902|   70.47542227327902| 70.47542227327902|      70.47542227327902|                  0.0|   70.47542227327902|
+-----------------+-------------------+---------------------+--------------------+------------------+-----------------------+---------------------+--------------------+



In [55]:
df_uv_uc =  df_user_course.join(
    df_flatten,
    on=['user_id'],
    how='left'
)
df_uv_uc.show()

+----------+---------+-----------+----------+----------+-------------+-------------------+----------+---------+-----------+---------+-----+-------------------+
|   user_id|course_id|certificate|start_date|  end_date|duration_days|        enroll_time|remain_day| video_id|start_point|end_point|speed|   local_start_time|
+----------+---------+-----------+----------+----------+-------------+-------------------+----------+---------+-----------+---------+-----+-------------------+
| U_1000902| C_696724|        1.0|2019-11-18|2020-02-29|        103.0|2019-12-29 20:40:49|      62.0|     NULL|       NULL|     NULL| NULL|               NULL|
| U_1000902| C_947149|        1.0|2020-04-20|2020-07-31|        102.0|2020-04-21 12:10:47|     101.0|     NULL|       NULL|     NULL| NULL|               NULL|
| U_1000902| C_676937|        0.0|2020-09-01|2020-12-31|        121.0|2020-10-14 16:46:34|      78.0|     NULL|       NULL|     NULL| NULL|               NULL|
|U_16685848| C_799796|       NULL|      

In [38]:
from pyspark.sql.functions import col

# Lọc df_grouped để chỉ lấy user U_30786396
user_U30786396_data = df_uv_uc.filter(col("user_id") == "U_30786396")

# Hiển thị toàn bộ thông tin của user này
# (thêm n=50 để hiển thị nhiều hàng hơn, truncate=False để xem đầy đủ)
print("Thông tin của User U_30786396:")
user_U30786396_data.orderBy('local_start_time').show(n=100, truncate=False)

Thông tin của User U_30786396:


+----------+---------+-----------+----------+----------+-------------+-------------------+----------+---------+-----------+---------+-----+-------------------+
|user_id   |course_id|certificate|start_date|end_date  |duration_days|enroll_time        |remain_day|video_id |start_point|end_point|speed|local_start_time   |
+----------+---------+-----------+----------+----------+-------------+-------------------+----------+---------+-----------+---------+-----+-------------------+
|U_30786396|C_629559 |NULL       |NULL      |NULL      |NULL         |2020-07-14 03:32:41|NULL      |V_1337580|4.0        |819.0    |1.0  |2020-08-31 11:45:03|
|U_30786396|C_1822804|1.0        |2020-06-05|2020-08-10|66.0         |2020-07-04 02:06:15|37.0      |V_1337580|4.0        |819.0    |1.0  |2020-08-31 11:45:03|
|U_30786396|C_676664 |1.0        |2020-07-14|2020-12-31|170.0        |2020-08-07 22:05:28|146.0     |V_1337580|4.0        |819.0    |1.0  |2020-08-31 11:45:03|
|U_30786396|C_707135 |0.0        |2020-0

In [27]:
df_uv_uc.count()

37695451

In [28]:
# 1. Lấy tổng số dòng
#total_row_count = df_uv_uc.count()

# 2. Tạo danh sách các biểu thức (expressions) để tính % null cho mỗi cột
#null_percentage_exprs = [
#    ((count(when(isnull(c), c)) / total_row_count) * 100).alias(f"perc_null_{c}")
#    for c in df_uv_uc.columns
#]

# 3. Select và show kết quả
# Kết quả sẽ là một hàng duy nhất hiển thị % null cho tất cả các cột
#df_uv_uc.select(null_percentage_exprs).show()

In [29]:
info_to_join = video_info
df_grouped = df_uv_uc.join(
    info_to_join,
    on='video_id',
    how='left'
)
#df_grouped.show()

In [30]:
##df_grouped.count().show()

In [31]:
# 1. Lấy tổng số dòng
total_row_count = df_grouped.count()

# 2. Tạo danh sách các biểu thức (expressions) để tính % null cho mỗi cột
null_percentage_exprs = [
   ((count(when(isnull(c), c)) / total_row_count) * 100).alias(f"perc_null_{c}")
   for c in df_grouped.columns
]

# 3. Select và show kết quả
# Kết quả sẽ là một hàng duy nhất hiển thị % null cho tất cả các cột
df_grouped.select(null_percentage_exprs).show()

+------------------+-----------------+-------------------+---------------------+--------------------+------------------+-----------------------+---------------------+--------------------+---------------------+-------------------+------------------+--------------------------+------------------+------------------------+
|perc_null_video_id|perc_null_user_id|perc_null_course_id|perc_null_certificate|perc_null_start_date|perc_null_end_date|perc_null_duration_days|perc_null_enroll_time|perc_null_remain_day|perc_null_start_point|perc_null_end_point|   perc_null_speed|perc_null_local_start_time|    perc_null_ccid|perc_null_video_duration|
+------------------+-----------------+-------------------+---------------------+--------------------+------------------+-----------------------+---------------------+--------------------+---------------------+-------------------+------------------+--------------------------+------------------+------------------------+
|16.512265631203086|              0.0|  

In [32]:
print(df_grouped.count())

37695451


In [33]:
from pyspark.sql import functions as F

df_grouped = df_grouped.withColumn(
    "day_from_enroll", 
    abs(col("day_from_enroll"))
)
df_grouped = df_grouped.drop("local_start_time","enroll_time")

TypeError: bad operand type for abs(): 'Column'

In [ ]:
df_grouped.show()

In [ ]:
from pyspark.sql import functions as F

df_grouped = (
    df_grouped
    # tính chỉ số phase 1..4, gộp tất cả >=4 vào 4
    .withColumn(
        "phase_index",
        F.when(F.col("day_from_enroll").isNull() | (F.col("day_from_enroll") < 0), None)
         .otherwise(
            F.least(F.floor(F.col("day_from_enroll") / 14) + 1, F.lit(4))
         )
    )
    # nhãn phase1..phase4
    .withColumn(
        "phase",
        F.when(F.col("phase_index").isNull(), None)
         .otherwise(F.concat(F.lit("phase"), F.col("phase_index").cast("string")))
    )
)
# kiểm tra nhanh
# df.select("day_from_enroll","phase_index","phase").orderBy("day_from_enroll").show(30, truncate=False)


In [ ]:
df_grouped.count()

In [ ]:
df_grouped.coalesce(1).write.mode("overwrite").option("header", "true").csv("/kaggle/working/df_group/")

In [ ]:
# import shutil

# input_dir = "/kaggle/working/df_group"
# output_zip = "/kaggle/working/df_group"

# shutil.make_archive(output_zip, 'zip', input_dir)
# print("✅ Đã nén file thành df_group.zip")


In [ ]:
# !rm -rf /kaggle/working/*
